In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://baloncestoenvivo.feb.es/resultados/ligaeba/8/2024"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    print(soup.prettify())  # Ver qué estructura tiene la página
else:
    print("Error al acceder a la página:", response.status_code)


<!DOCTYPE html>
<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Competiciones FEB
  </title>
  <meta content="initial-scale=1.0, user-scalable=no" name="viewport"/>
  <script src="https://baloncestoenvivo.feb.es/scripts/jquery-1.11.0.js">
  </script>
  <script src="https://baloncestoenvivo.feb.es/scripts/basics.js?v=23092020">
  </script>
  <!--<link href="https://baloncestoenvivo.feb.es/styles/theme.css?v=02102020" rel="stylesheet" />-->
  <link href="https://baloncestoenvivo.feb.es/styles/theme.css?v=30122024" rel="stylesheet"/>
  <link href="https://www.feb.es/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <script src="https://baloncestoenvivo.feb.es/scripts/handlebars.js">
  </script>
  <script src="https://baloncestoenvivo.feb.es/scripts/widget.launcher.js">
  </script>
  <script src="https://baloncestoenvivo.feb.es/scripts/widget.handlebars.helpers.js">
  </script>
  <script src="https://baloncestoenvivo.feb.es/scripts/

In [16]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import timedelta

def obtener_equipos(url):
    respuesta = requests.get(url)
    soup = BeautifulSoup(respuesta.text, 'html.parser')
    
    equipos = []
    for fila in soup.select('#_ctl0_MainContentPlaceHolderMaster_clasificacionDataGrid tr')[1:]:  #Vamos a la tabla de clasificación para obtener los equipos
        enlace = fila.find('a')
        if enlace:
            nombre = enlace.text.strip()
            url_equipo = enlace['href']
            equipos.append((nombre, url_equipo))
    return equipos

def mostrar_menu(equipos):
    print("Selecciona un equipo para analizar:")
    for i, (nombre, _) in enumerate(equipos, start=1):
        print(f"{i}. {nombre}")
    
    eleccion = int(input("Ingresa el número del equipo: ")) - 1
    return equipos[eleccion]



def obtener_id(url_equipo):
    codigo_equipo = url_equipo.split("i=")[-1]
    return codigo_equipo


def obtener_estadisticas(url):
    # Realizamos la solicitud a la página
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Inicializamos una lista para almacenar los datos de los jugadores
        jugadores_data = []

        # Buscamos todas las filas de la tabla con las estadísticas (ignoramos la cabecera)
        rows = soup.find_all("tr")[3:]  # Omite las primeras filas (cabecera y otras)

        for row in rows:
            cols = row.find_all("td")

            if len(cols) > 0:  # Verifica que la fila tenga columnas
                nombre_jugador = cols[0].get_text(strip=True)
                enlace_jugador = cols[0].find("a")["href"] if cols[0].find("a") else None
                fase = cols[1].get_text(strip=True)
                partidos = cols[2].get_text(strip=True)
                minutos = cols[3].get_text(strip=True)
                puntos = cols[4].get_text(strip=True)
                tiros_dos = cols[5].get_text(strip=False).split()[0]
                tiros_tres = cols[6].get_text(strip=False).split()[0]
                tiros_campo = cols[7].get_text(strip=False).split()[0]
                tiros_libres = cols[8].get_text(strip=False).split()[0]
                rebotes_ofensivos = cols[9].get_text(strip=True)
                rebotes_defensivos = cols[10].get_text(strip=True)
                rebotes_totales = cols[11].get_text(strip=True)
                asistencias = cols[12].get_text(strip=True)
                recuperaciones = cols[13].get_text(strip=True)
                perdidas = cols[14].get_text(strip=True)
                tapones_favor = cols[15].get_text(strip=True)
                tapones_contra = cols[16].get_text(strip=True)
                mates = cols[17].get_text(strip=True)
                faltas_cometidas = cols[18].get_text(strip=True)
                faltas_recibidas = cols[19].get_text(strip=True)
                valoracion = cols[20].get_text(strip=True)

                # Añadimos los datos de la fila a la lista de jugadores
                jugadores_data.append([
                    nombre_jugador, 
                    enlace_jugador, 
                    fase, 
                    partidos, 
                    minutos, 
                    puntos, 
                    tiros_dos, 
                    tiros_tres, 
                    tiros_campo, 
                    tiros_libres, 
                    rebotes_ofensivos, 
                    rebotes_defensivos, 
                    rebotes_totales, 
                    asistencias, 
                    recuperaciones, 
                    perdidas, 
                    tapones_favor, 
                    tapones_contra, 
                    mates, 
                    faltas_cometidas, 
                    faltas_recibidas, 
                    valoracion
                ])

        # Creamos el DataFrame
        df = pd.DataFrame(jugadores_data, columns=[
            "Jugador", "Enlace", "Fase", "Partidos", "Minutos", "Puntos", "T2", "T3", "TC", "TL", 
            "RO", "RD", "RT", "AS", "BR", "BP", "TF", "TC", "MT", "FC", "FR", "Valoración"
        ])
        #print(df)
        return df

    else:
        print(f"Error al obtener las estadísticas: {response.status_code}")
        return None
    
def calcular_usage_rate(df, total_team_stats):
    """
    Calcula el porcentaje de uso (USG%) de cada jugador en base a las estadísticas del equipo.
    :param df: DataFrame con estadísticas individuales
    :param total_team_stats: Diccionario con estadísticas totales del equipo
    :return: DataFrame con la columna USG% añadida
    """
    team_fga = total_team_stats['FGA']
    print(team_fga)
    team_fta = total_team_stats['FTA']
    team_tov = total_team_stats['TOV']
    team_minutes = total_team_stats['Minutos']

    # Calcular USG% para cada jugador
    df['USG%'] = 100 * (
        (df['T2'].apply(lambda x: int(x.split('/')[1])) +  # Intentos de 2pts
        df['T3'].apply(lambda x: int(x.split('/')[1])) +  # Intentos de 3pts
        0.44 * df['TL'].apply(lambda x: int(x.split('/')[1])) +  # Intentos de TL ajustados
        df['BP'].astype(int)) * (team_minutes / 5)
    ) / (
        df['Minutos'].astype(int) * (team_fga + 0.44 * team_fta + team_tov)
    )

    return df





    

In [ ]:
def main():
    url_feb = "https://baloncestoenvivo.feb.es/resultados/ligaeba/8/2024"
    equipos = obtener_equipos(url_feb)
    nombre_seleccionado, url_equipo = mostrar_menu(equipos)
    print(f"Has seleccionado: {nombre_seleccionado}")
    print(f"URL del equipo: {url_equipo}")

    id_equipo = obtener_id(url_equipo)
    url_estadisticas = f"https://baloncestoenvivo.feb.es/estadisticasacumuladas/{id_equipo}"
    estadisticas=obtener_estadisticas(url_estadisticas)
    print("Estadisticas obtenidas")
    display(estadisticas)

    # Extraer la última fila como los totales del equipo
    totales_equipo = estadisticas.iloc[-1]
    estadisticas = estadisticas.iloc[:-1]

    
   


    # Convertir a un diccionario de valores clave
    estadisticas_totales = {
        'FGA': int(totales_equipo['T2'].split('/')[1]) + int(totales_equipo['T3'].split('/')[1]),
        'FTA': int(totales_equipo['TL'].split('/')[1]),
        'TOV': int(totales_equipo['BP']),
        'Minutos': 4400 #Para esto, se me ocurre coger en el web scrapping los partidos jugados del equipo(se puede ver en la
        #clasificación ) y multiplicarlo por 200.
    }
    print(estadisticas_totales)

    
    estadisticas_avanzadas=calcular_usage_rate(estadisticas, estadisticas_totales)



if __name__ == "__main__":
    main()

Selecciona un equipo para analizar:
1. ESPORTIU BÀSQUET VILA-REAL
2. NB TORRENT
3. C.B. PUERTO SAGUNTO
4. EL PILAR- UPV
5. JOVENS L’ELIANA
6. VALENCIA B.C.
7. TOPSURFACE NB PATERNA
8. RICASA GODELLA
9. CB JOVENS ALMÀSSERA
10. AMICS CASTELLÓ B
11. MARISTAS VALENCIA
12. CB TABERNES BLANQUES - FERNANDO GIL
13. CB MORVEDRE
14. PICKEN CLARET
Has seleccionado: NB TORRENT
URL del equipo: https://baloncestoenvivo.feb.es/Equipo.aspx?i=951964
Estadisticas obtenidas


,Jugador,Enlace,Fase,Partidos,Minutos,Puntos,T2,T3,TC,TL,...,RT,AS,BR,BP,TF,TC,MT,FC,FR,Valoración
0,"ALBEROLA GUILLOT, MARC",https://baloncestoenvivo.feb.es/Jugador.aspx?i...,LR,22,531:08,168,35/91,12/48,47/139,62/94,...,78,85,40,52,3,4,0,52,90,236
1,"BEDIA FERRER, JOSE ANTONIO",https://baloncestoenvivo.feb.es/Jugador.aspx?i...,LR,21,481:23,171,70/105,1/4,71/109,28/44,...,156,16,31,14,13,4,1,47,31,303
2,"BLASCO NAVARRO, LUCAS",https://baloncestoenvivo.feb.es/Jugador.aspx?i...,LR,3,67:01,36,13/21,1/4,14/25,7/10,...,24,3,3,4,0,0,0,0,8,56
3,"BOSCH ROIG, JOAN",https://baloncestoenvivo.feb.es/Jugador.aspx?i...,LR,18,284:21,87,27/66,6/20,33/86,15/28,...,99,15,7,20,4,4,2,24,21,123
4,"CALERO BAUTISTA, DAVID",https://baloncestoenvivo.feb.es/Jugador.aspx?i...,LR,17,299:39,155,42/91,14/53,56/144,29/53,...,59,10,9,18,6,1,2,33,37,113
5,"CAMPILLOS ALONSO, CARLOS",https://baloncestoenvivo.feb.es/Jugador.aspx?i...,LR,21,513:42,198,22/63,38/125,60/188,40/50,...,53,71,18,44,5,1,0,33,46,176
6,"GARCIA MEDINA, MIGUEL",https://baloncestoenvivo.feb.es/Jugador.aspx?i...,LR,1,00:46,3,1/1,0/0,1/1,1/1,...,0,0,0,0,0,0,0,0,1,4
7,"GONZÁLEZ JURADO, ÓSCAR",https://baloncestoenvivo.feb.es/Jugador.aspx?i...,LR,1,01:44,3,0/0,1/1,1/1,0/0,...,0,0,0,0,0,0,0,0,0,3
8,"HERNANDEZ BENACHES, SERGIO",https://baloncestoenvivo.feb.es/Jugador.aspx?i...,LR,20,397:46,156,17/34,35/116,52/150,17/20,...,62,31,26,33,0,2,1,23,27,145
9,"LOPEZ DOMINGUEZ, PABLO",https://baloncestoenvivo.feb.es/Jugador.aspx?i...,LR,22,503:44,190,54/125,16/71,70/196,34/58,...,108,47,24,37,6,7,1,48,54,194


{'FGA': 1544, 'FTA': 540, 'TOV': 279, 'Minutos': 4400}
1544


ValueError: invalid literal for int() with base 10: '531:08'